# Full Workflow

Feature Extraction + Word embedding + Clustering + Evaluation

# 1. Feature Extraction

In [1]:
import pandas as pd
import numpy as np
import nltk
import regex
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import stanza
stanza.download('en') # download English model
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

2021-03-25 23:44:52 INFO: Downloading default packages for language: en (English)...
2021-03-25 23:44:54 INFO: File exists: C:\Users\TzeMin\stanza_resources\en\default.zip.
2021-03-25 23:45:00 INFO: Finished downloading models and saved to C:\Users\TzeMin\stanza_resources.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TzeMin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TzeMin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\TzeMin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def feature_extraction(txt, nlp):
    
    sentList = nltk.sent_tokenize(txt)
    retlist = [];
    
    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
        newwordList = []
        flag = 0
        
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])
        finaltxt = ' '.join(word for word in newwordList)
    
        stop_words = set(stopwords.words('english'))
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)
        
        doc = nlp(finaltxt)
        dep_node = []
        try:
            for dep_edge in doc.sentences[0].dependencies:
                dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
            for i in range(0, len(dep_node)):
                if (int(dep_node[i][1]) != 0):
                    dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
        except:
            pass;
                
        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i))
                categories.append(i[0])        
        
        fcluster = []
        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in [
                    # Different types of words that are identified as potential features
                    "nsubj",
                    #"acl:relcl",
                    "obj",
                    "dobj",
                    #"agent",
                    #"advmod",
                    #"amod",
                    #"neg",
                    #"prep_of",
                    #"acomp",
                    #"xcomp",
                    #"compound"
                ])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
        print(fcluster) 
        
        # Remove all features with no sentiment word:
        retlist.append(fcluster)
    return retlist;

def do_extraction(df, nlp, feat_count, feat_sent, content_str = "Content"):
    idx = 0;
    # Replace "" with nan's for removal
    df[content_str].replace('', np.nan, inplace=True)
    df.dropna(subset=[content_str], inplace=True)
    review_list = df[content_str].to_list()     
    print(" Processing : " , df.shape[0], "rows of data")
    
    for review in tqdm(review_list):
        print("Review Number : ", idx);
        
        # Some data pre-processing
        review = review.lower()
        
        # Merge hyphenated words
        separate = review.split('-')
        review = ''.join(separate)
        
        # Remove non-alphabets
        review = re.sub(r'[^a-z\s\t]', '', review)
        
        idx += 1;
        if idx >= df.shape[0]:
            break;
        try:
            output = feature_extraction(review, nlp);
        except:
            pass;
        for sent in output:
            for pair in sent:
                print(pair)
                if pair[0] in feat_sent:
                    if pair[1] is not None:
                        flist = feat_sent[pair[0]]
                        if isinstance(pair[1], list):
                            for i in pair[1]:
                                flist.append(i)
                        else:
                            flist.append(pair[1])
                        feat_sent[pair[0]] = flist;
                else:
                    if pair[1] is not None:
                        flist = pair[1]
                    else:
                        flist = list()
                    feat_sent[pair[0]] = flist;
                
                if pair[0] in feat_count:
                    feat_count[pair[0]] = feat_count[pair[0]] + 1;
                else:
                    feat_count[pair[0]] = 1
    
    return feat_count, feat_sent;

def get_sentiment(feat_count, feat_sent, nlp):

    sentiment_score = dict()

    # Delete features with no descriptors
    cob = feat_sent.copy()
    for feat in cob.keys():
        if cob[feat] == []:
            del feat_sent[feat]
        else:
            feat_sent[feat] = ' ,'.join(feat_sent[feat])

    # Run pre-built sentiment score and take avg of all descriptors
    for f in tqdm(feat_sent.keys()):
        print("Calculating Sentiment for: ", f);
        ssum = 0;
        for g in feat_sent[f]:
            try:
                doc = nlp(g);
                for i in doc.sentences:
                        ssum += i.sentiment;
            except:
                pass;
        sentiment_score[f] = ssum / len(b[f])

    adf = pd.DataFrame.from_dict(feat_count, orient='index', columns=['Freq'])
    adf.sort_values(by="Freq", ascending=False, inplace = True)

    avg_sent = pd.DataFrame.from_dict(sentiment_score, orient='index', columns=["Avg_sent"])
    desc_words = pd.DataFrame.from_dict(feat_sent, orient="index", columns=["Descriptors"])
    
    avg_sent = avg_sent.merge(desc_words, left_index=True, right_index=True)
    final_sent = avg_sent.merge(adf, left_index=True, right_index=True)
    final_sent.sort_values(by="Freq", ascending=False, inplace=True)
    return final_sent;

In [3]:
rdr = pd.read_csv('../../output/scraped-ns/cmpb.csv')

nlp = stanza.Pipeline('en')
a = dict()
b = dict()
a, b = do_extraction(rdr, nlp, a, b)

2021-03-25 23:45:00 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-25 23:45:00 INFO: Use device: cpu
2021-03-25 23:45:00 INFO: Loading: tokenize
2021-03-25 23:45:00 INFO: Loading: pos
2021-03-25 23:45:01 INFO: Loading: lemma
2021-03-25 23:45:01 INFO: Loading: depparse
2021-03-25 23:45:01 INFO: Loading: sentiment
2021-03-25 23:45:02 INFO: Loading: ner
2021-03-25 23:45:03 INFO: Done loading processors!
  0%|                                                                                           | 0/52 [00:00<?, ?it/s]

 Processing :  52 rows of data
Review Number :  0


  2%|█▌                                                                                 | 1/52 [00:00<00:29,  1.74it/s]

[['professional', []], ['people', ['recommended']], ['patient', []], ['kind', ['recommended']], ['respectful', []], ['smooth', []], ['medical', []], ['checkup', []]]
['professional', []]
['people', ['recommended']]
['patient', []]
['kind', ['recommended']]
['respectful', []]
['smooth', []]
['medical', []]
['checkup', []]
Review Number :  1


  4%|███▏                                                                               | 2/52 [00:01<00:26,  1.88it/s]

...

[['place', []], ['well', []], ['people', ['kept']], ['unbelievably', []]]
['place', []]
['well', []]
['people', ['kept']]
['unbelievably', []]
Review Number :  46
[['bad', []], ['service', []]]
['bad', []]
['service', []]
Review Number :  47


 94%|█████████████████████████████████████████████████████████████████████████████▎    | 49/52 [00:18<00:00,  5.35it/s]

[['interestingly', []], ['enough', []], ['negative', []], ['reviews', []]]
['interestingly', []]
['enough', []]
['negative', []]
['reviews', []]
Review Number :  48
[['bad', []], ['service', []]]
['bad', []]
['service', []]
Review Number :  49


 98%|████████████████████████████████████████████████████████████████████████████████▍ | 51/52 [00:19<00:00,  2.67it/s]

[['sheat', []], ['dirty', []], ['pigs', []], ['step', []], ['minefields', []]]
['sheat', []]
['dirty', []]
['pigs', []]
['step', []]
['minefields', []]
Review Number :  50
[['gncpresent', []]]
['gncpresent', []]
Review Number :  51


In [4]:
fin = get_sentiment(a, b, nlp)

  0%|                                                                                           | 0/89 [00:00<?, ?it/s]

Calculating Sentiment for:  professional


  1%|▉                                                                                  | 1/89 [00:00<01:17,  1.13it/s]

Calculating Sentiment for:  people


  2%|█▊                                                                                 | 2/89 [00:05<04:35,  3.16s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████▏ | 87/89 [00:55<00:00,  2.04it/s]

Calculating Sentiment for:  money


 99%|█████████████████████████████████████████████████████████████████████████████████ | 88/89 [00:55<00:00,  2.46it/s]

Calculating Sentiment for:  attitude


100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:56<00:00,  1.58it/s]


In [5]:
fin

,Avg_sent,Descriptors,Freq
staff,0.884058,"professional ,keep ,keep ,rude ,impatient ,exp...",11
people,0.891892,"recommended ,suggest ,threatens ,threatens ,th...",10
bad,0.913043,"its ,canteenb ,canteenb",8
reviews,0.894737,"suggest ,what ,give",6
place,0.903226,"clean ,experience ,waste ,check",6
...,...,...,...
train,1.000000,army,1
army,1.000000,train,1
directions,1.000000,seen,1
process,1.000000,expect,1


In [6]:
def get_tfidf_features(df, content_str = "Content", min_ = 2, max_ = 0.5, ngramrange = (1,2)):
    review_list = df[content_str].to_list()
    tfidf = TfidfVectorizer(min_df = min_, max_df = max_, ngram_range = ngramrange);
    features = tfidf.fit_transform(review_list);
    q = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())
    return list(q.columns)

def refine_features(originaldf, sentimentdf):
    tfidf_output = get_tfidf_features(originaldf)
    sentimentdf = sentimentdf.reset_index()
    ft_extract = set(sentimentdf['index']);
    tfidf_extract = set(tfidf_output)
    
    intersecting_features = ft_extract.intersection(tfidf_extract)
    
    return_df = sentimentdf
    return_df = return_df.loc[return_df['index'].isin(list(intersecting_features))]
    print("Number of extracted features:")
    print("Initial = ", len(ft_extract), " TFIDF = ", len(intersecting_features), " Final after intersection = ", return_df.shape[0])
    return return_df

In [7]:
refined = refine_features(rdr, fin)
refined

Number of extracted features:
Initial =  89  TFIDF =  31  Final after intersection =  31


,index,Avg_sent,Descriptors,Freq
0,staff,0.884058,"professional ,keep ,keep ,rude ,impatient ,exp...",11
1,people,0.891892,"recommended ,suggest ,threatens ,threatens ,th...",10
2,bad,0.913043,"its ,canteenb ,canteenb",8
3,reviews,0.894737,"suggest ,what ,give",6
4,place,0.903226,"clean ,experience ,waste ,check",6
5,time,0.892857,"takes ,takes ,travel ,travel",5
6,rude,1.000000,staff,4
7,experience,0.900000,"place ,had",4
8,nsf,0.900000,"need ,cool",4
9,professional,0.916667,"staff ,medic",3


# 2. Word Embedding + Clustering

Do some cleaning first. From the set of refined features, we removed stop words, then stemmed them.

In [8]:
import collections
from sklearn.cluster import KMeans
from tqdm import tqdm
from gensim.models import Word2Vec 
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
import spacy
from nltk.stem import PorterStemmer

In [9]:
uncleaned_words = refined['index']
uncleaned_words.head()

0      staff
1     people
2        bad
3    reviews
4      place
Name: index, dtype: object

In [10]:
nlp = spacy.load("en_core_web_sm") # to run on command prompt: python -m spacy download en_core_web_sm
words = [item for item in uncleaned_words if item not in nlp.Defaults.stop_words]
print("Words removed were: ", set(uncleaned_words).difference(set(words)))
print("From", len(uncleaned_words), "to", len(words))
words_df = pd.DataFrame(words)
words_df.columns = ['word']
words_df

Words removed were:  {'take', 'others'}
From 31 to 29


,word
0,staff
1,people
2,bad
3,reviews
4,place
5,time
6,rude
7,experience
8,nsf
9,professional


In [32]:
stemmed_words = []
ps = PorterStemmer()

for w in words:
    rootWord = ps.stem(w)
    if rootWord not in stemmed_words:
        stemmed_words.append(rootWord)
        
print("Number of words left after stemming was", len(stemmed_words))

Number of words left after stemming was 29


## 2.1 spaCy's Pretained Vectors + Affinity Propagation

In [13]:
def vectorize(text):
    """Get the SpaCy vector corresponding to a text"""
    return nlp(text).vector

X = np.stack(vectorize(word) for word in words)
X_normalised = normalize(np.stack(vectorize(word) for word in words))

affprop = AffinityPropagation()
affprop.fit(X)

word_array = np.array(words)

c:\users\tzemin\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3337: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\tzemin\appdata\local\programs\python\python37\lib\site-packages\sklearn\cluster\_affinity_propagation.py:154: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning


In [14]:
for cluster_id in np.unique(affprop.labels_):
    exemplar = word_array[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(word_array[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *place:* attitude, cmpb, experience, nsf, patience, person, place, question, staff, time, tone
 - *check:* ask, check, help, rest, waste
 - *guards:* guards, hours, people, reviews
 - *nice:* bad, complete, friendly, good, great, kind, nice, professional, rude


# Evaluation

In [23]:
from sklearn import metrics

labels_true_df = pd.read_csv("../../output/evaluation/cluster_manual.csv")
labels_true_df

,Cluster,Elements
0,1,staff
1,1,person
2,1,personal
3,1,guards
4,1,doctors
...,...,...
192,148,whole
193,149,process
194,150,take
195,151,august


In [24]:
labels_true_df.columns = ["true_cluster", "word"]
labels_true_df = pd.merge(labels_true_df, words_df, on = "word", how = "inner")
labels_true_df

,true_cluster,word
0,1,staff
1,1,person
2,1,guards
3,3,place
4,4,time
5,4,hours
6,5,rude
7,5,friendly
8,5,kind
9,6,good


In [25]:
set(words_df['word']).difference(set(labels_true_df['word']))

set()

In [26]:
print(affprop.labels_)
labels_pred_df = pd.DataFrame({'pred_cluster': affprop.labels_})
labels_pred_df
pd.merge(labels_true_df, labels_pred_df, left_index = True, right_index = True)

[0 2 3 2 0 0 3 0 0 3 0 1 3 3 1 1 0 2 3 3 0 3 0 0 2 1 1 3 0]


,true_cluster,word,pred_cluster
0,1,staff,0
1,1,person,2
2,1,guards,3
3,3,place,2
4,4,time,0
5,4,hours,0
6,5,rude,3
7,5,friendly,0
8,5,kind,0
9,6,good,3


In [30]:
labels_true = list(labels_true_df['true_cluster'])

## Rand Index

In [31]:
metrics.rand_score(labels_true, affprop.labels_)

0.7167487684729064

In [34]:
metrics.adjusted_rand_score(labels_true, affprop.labels_)

-0.020635683994229003

## Mutual Information based scores

In [33]:
metrics.adjusted_mutual_info_score(labels_true, affprop.labels_)

-0.0435904889302693

## Silhouette score

In [36]:
metrics.silhouette_score(X, affprop.labels_, metric='euclidean')

0.21163464

## Calinski-Harabasz Index

In [37]:
metrics.calinski_harabasz_score(X, affprop.labels_)

8.084159232372821

## Davies Bouldin Index

In [39]:
metrics.davies_bouldin_score(X, affprop.labels_)

1.4980370150395048